# Marcaje de entidades

In [1]:

import json
import spacy
import numpy as np
import pandas as pd
from collections import Counter
from tqdm import tqdm

In [ ]:
#ver el dataset preprosesado
import pandas as pd
df = pd.read_csv('/Users/fjosesala/Documents/GitHub/IBD-NLP-RiskPrediction/data/processed/reddit_ibd_preprocessed.csv')
df.head()

## Marcaje de entidades con Spacy

In [38]:
import spacy
import pandas as pd
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [ ]:
# Cargar dataset
df = pd.read_csv("/content/drive/Shareddrives/IBD-NLP-RiskPrediction_data/Reddit/processed/reddit_ibd_preprocessed.csv")
df.head()

,id,subreddit,author,title,selftext,created_utc,comments,cuerpo,title_clean,title_lemma,selftext_clean,selftext_lemma,comments_clean,comments_lemma
0,dflwn,CrohnsDisease,zakool21,Don't be afraid of diagnostic procedures....,"I'm not likely to frontpage this subreddit, bu...",2010-09-18 11:59:11,"['I have to agree, the worse part of a Colonos...",TÍTULO:\nDon't be afraid of diagnostic procedu...,do not be afraid of diagnostic procedures,afraid diagnostic procedure,i am not likely to frontpage this subreddit bu...,likely frontpage subreddit glad exist want rea...,i have to agree the worse part of a colonoscop...,agree bad colonoscopy prep glad ok case thank ...
1,dfyy1,CrohnsDisease,sphinctersayzwha,Mayo Clinic article on Crohn's Disease.,Mayo Clinic article on Crohn's Disease.,2010-09-19 15:11:21,[],TÍTULO:\nMayo Clinic article on Crohn's Diseas...,mayo clinic article on crohn s disease,mayo clinic article crohn s disease,mayo clinic article on crohn s disease,mayo clinic article crohn s disease,NaN,NaN
2,dfz1m,CrohnsDisease,WeDeserveDessert,Has anyone else here taken Remicade? What did...,Has anyone else here taken Remicade? What did...,2010-09-19 15:19:54,"[""Remicade (and a bypass) changed my life. Wh...",TÍTULO:\nHas anyone else here taken Remicade? ...,has anyone else here taken remicade what did y...,take remicade think help,has anyone else here taken remicade what did y...,take remicade think help,remicade and a bypass changed my life while it...,remicade bypass change life eventually decreas...
3,dh6zd,CrohnsDisease,RosenTurd,"I was Diagnosed With Crohns Disease at age 9 ,...",Like the title Says AMA,2010-09-22 04:50:57,"[""My son was diagnosed at ten. It's hell for h...",TÍTULO:\nI was Diagnosed With Crohns Disease a...,i was diagnosed with crohns disease at age 9 n...,diagnose crohns disease age 9 28yo ama,like the title says ama,like title say ama,my son was diagnosed at ten it is hell for him...,son diagnose hell colonoscopy hospital visit d...
4,dhgot,CrohnsDisease,unknownpleasures,Has anyone here had to have bowel surgery more...,I had a small bowel resection and appendectomy...,2010-09-22 18:31:36,"[""I have a story:\n\nI had a small bowel resec...",TÍTULO:\nHas anyone here had to have bowel sur...,has anyone here had to have bowel surgery more...,bowel surgery,i had a small bowel resection and appendectomy...,small bowel resection appendectomy 6 year ago ...,i have a story n ni had a small bowel resectio...,story n ni small bowel resection appendectomy ...


In [ ]:
# Cargar modelo spaCy
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Procesar textos
texts = df["selftext_lemma"].fillna("").tolist()

ner_results = []

for text in tqdm(texts, desc="Applying NER"):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    ner_results.append(entities)

df["entities"] = ner_results

# Guardar dataset con entidades
df.to_csv("/content/drive/Shareddrives/IBD-NLP-RiskPrediction_data/Reddit/processed/reddit_ibd_with_entities.csv", index=False)

Applying NER: 100%|██████████| 183744/183744 [33:18<00:00, 91.96it/s]


In [ ]:
from collections import Counter
import itertools

all_entities = list(itertools.chain.from_iterable(df["entities"]))
labels = [label for _, label in all_entities]

Counter(labels).most_common(10)

[('DATE', 241861),
 ('CARDINAL', 103328),
 ('ORG', 52006),
 ('TIME', 30544),
 ('PERSON', 23558),
 ('ORDINAL', 12835),
 ('GPE', 9411),
 ('QUANTITY', 8701),
 ('NORP', 6550),
 ('PRODUCT', 3017)]

In [ ]:
# Ejemplo con entidades resaltadas
import random
from spacy import displacy

i = random.randint(0, len(df)-1)
doc = nlp(df.iloc[i]["selftext_lemma"])
displacy.render(doc, style="ent", jupyter=True)

## Marcaje de entidades con ScySpacy

In [ ]:
%pip install scispacy
%pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_md-0.5.4.tar.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.1/119.1 MB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.0/865.0 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 116.1 MB/s eta 0:00:00
  Created wheel for en_core_sci_md: filename=en_core_sci_md-0.5.4-py3-none-any.whl size=119157918 sha256=8192229d0456faa8f5978b455764516ac335ca1bd9ac67d15f7ca4dda826d8f8
  Stored in directory: /root/.cache/pip/wheels/1a/02/d9/4d4bda80f6b73c02ed057d4b3b99abaff286924a80b53c9d68
Successfully built en_core_sci_md
  Attempting uninstall: blis
    Found existing installation: blis 1.2.1
    Uninstalling blis-1.2.1:
      Successfully uninstalled blis-1.2.1
  Attempting uninstall: thinc
    Found existing installation: thinc 8.3.4
    Uninstalling thinc-8.3.4:
      Successfully uninstalled thinc-8.3.4
  Attempting uninstall: spacy
    Fou

In [ ]:
# Cargar modelo spaCy
nlp = spacy.load("en_core_sci_md")

/usr/local/lib/python3.12/dist-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


In [ ]:
# Procesar textos
texts = df["selftext_lemma"].fillna("").tolist()

ner_results = []

for text in tqdm(texts, desc="Applying NER"):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    ner_results.append(entities)

df["entities"] = ner_results

# Guardar dataset con entidades
df.to_csv("/content/drive/Shareddrives/IBD-NLP-RiskPrediction_data/Reddit/processed/reddit_ibd_with_entities_scispacy.csv", index=False)

Applying NER: 100%|██████████| 183744/183744 [37:16<00:00, 82.17it/s]


In [ ]:
from collections import Counter
import itertools

all_entities = list(itertools.chain.from_iterable(df["entities"]))
labels = [label for _, label in all_entities]

Counter(labels).most_common(10)

[('ENTITY', 2331327)]

In [ ]:
# Ejemplo con entidades resaltadas
import random
from spacy import displacy

i = random.randint(0, len(df)-1)
doc = nlp(df.iloc[i]["selftext_lemma"])
displacy.render(doc, style="ent", jupyter=True)

## Marcaje de entidades con Bert y Biobert Finetuneado

In [ ]:
# Analisis para ver si truncar los elementos para bert y biobert funciona para nuestro dataset (estos modelos tienen un limite de tokens)

import pandas as pd
df = pd.read_csv('/Users/fjosesala/Documents/GitHub/IBD-NLP-RiskPrediction/data/processed/reddit_ibd_preprocessed.csv')
df['text_length'] = df['selftext_lemma'].str.len()

print(f"Mediana: {df['text_length'].median():.0f} chars")
print(f"75 percentil: {df['text_length'].quantile(0.75):.0f} chars")
print(f"95 percentil: {df['text_length'].quantile(0.95):.0f} chars")
print(f"Textos >2000: {(df['text_length'] > 2000).sum()} ({(df['text_length'] > 2000).sum()/len(df)*100:.1f}%)")


    
#si se puede truncar, no hay mayor perdida de infomracion


In [ ]:
#Comparacion de modelos NER: BERT, BioBERT, spaCy, scispaCy, con el cuerpo de texto truncado a 2000 caracteres para evitar errores de longitud. LEMATIZADO


import torch
import json
from transformers import pipeline
import spacy
import numpy as np
import pandas as pd
from collections import Counter
from tqdm import tqdm

print(f"✅ PyTorch: {torch.__version__}")

# Modelos
HF_MODELS = {
    "bert": "dslim/bert-base-NER",
    "biobert": "dmis-lab/biobert-v1.1",  # BioBERT base (sin fine-tune específico NER)
    "biobert": "alvaroalon2/biobert_diseases_ner"  # Fine-tuned en enfermedades
    "biobert": "siddharthtumre/biobert-finetuned-ner"
}


# spaCy
nlp_spacy = spacy.load("en_core_web_sm")
nlp_scispacy = None  # Omitir por ahora

# Helpers
def normalize_hf(ner_out):
    spans = []
    for ent in ner_out:
        label = ent.get("entity_group") or ent.get("entity")
        spans.append((ent["start"], ent["end"], label, ent["word"]))
    return spans

def normalize_spacy(doc):
    return [(ent.start_char, ent.end_char, ent.label_, ent.text) for ent in doc.ents]

def truncate_text(text, max_chars=2000):
    """Trunca a ~512 tokens (aprox 2000 chars)"""
    return text[:max_chars] if len(text) > max_chars else text

# Cargar datos
df = pd.read_csv('/Users/fjosesala/Documents/GitHub/IBD-NLP-RiskPrediction/data/processed/reddit_ibd_preprocessed.csv')
texts = df["selftext_lemma"].fillna("").astype(str).tolist()
ids = df.get("id", pd.Series(range(len(df)))).tolist()

print(f"Total textos: {len(texts)}")

# Crear pipelines (sin truncation en constructor)
hf_pipes = {}
for name, model_name in HF_MODELS.items():
    print(f"Cargando {name}...")
    hf_pipes[name] = pipeline("ner", model=model_name, tokenizer=model_name, grouped_entities=True, device=-1)

print("✅ Modelos cargados\n")

# Procesar (prueba con 100 primero)
SAMPLE = len(texts)
idxs = list(range(min(SAMPLE, len(texts))))
results = []

for i in tqdm(idxs, desc="NER"):
    text_orig = texts[i]
    text = truncate_text(text_orig, max_chars=2000)  # ← Trunca aquí
    
    row = {"id": ids[i], "text": text, "text_original_length": len(text_orig)}
    
    # HF models
    for name, pipe in hf_pipes.items():
        try:
            out = pipe(text)
            row[f"entities_{name}"] = normalize_hf(out)
        except Exception as e:
            print(f"⚠️  Error {name} en texto {i}: {str(e)[:50]}")
            row[f"entities_{name}"] = []
    
    # spaCy (también puede fallar con textos largos)
    try:
        if len(text) < 1000000:  # spaCy tiene límite 1M chars
            doc_sp = nlp_spacy(text)
            row["entities_spacy"] = normalize_spacy(doc_sp)
        else:
            row["entities_spacy"] = []
    except Exception as e:
        print(f"⚠️  Error spaCy en texto {i}")
        row["entities_spacy"] = []
    
    # scispaCy (omitir por ahora)
    row["entities_scispacy"] = []
    
    results.append(row)

# Guardar
out_df = pd.DataFrame(results)

import os
os.makedirs("./data/processed", exist_ok=True)

out_df.to_pickle("./data/processed/ner_comparisons.pkl")
out_df.to_csv("./data/processed/ner_comparisons_preview.csv", index=False)

print(f"\n✅ Procesados {len(results)} textos")
print(f"📁 Archivos guardados en ./data/processed/")

# Resumen rápido
for model in ["bert", "biobert", "spacy"]:
    col = f"entities_{model}"
    total = out_df[col].apply(len).sum()
    print(f"  {model}: {total} entidades detectadas")


✅ PyTorch: 2.9.0


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Total textos: 183744
Cargando bert...


Device set to use cpu
/Users/fjosesala/Documents/GitHub/IBD-NLP-RiskPrediction/.venv/lib/python3.13/site-packages/transformers/pipelines/token_classification.py:186: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


Cargando biobert...


Device set to use cpu


✅ Modelos cargados



NER: 100%|██████████| 183744/183744 [4:22:46<00:00, 11.65it/s]  



✅ Procesados 183744 textos
📁 Archivos guardados en ./data/processed/
  bert: 1355 entidades detectadas
  biobert: 860851 entidades detectadas
  spacy: 486119 entidades detectadas


In [ ]:
# Checkear resultados generados por cada modelo NER

import pandas as pd
from collections import Counter

# Cargar resultados
df_results = pd.read_pickle("./data/processed/ner_comparisons.pkl")

print(f"Total de textos procesados: {len(df_results)}")
print(f"="*80)

# Ver varios ejemplos para comparar
for idx in [0, 1, 2, 4, 10]:
    print(f"\n{'='*80}")
    print(f"EJEMPLO {idx}")
    print(f"{'='*80}")
    
    text = df_results.loc[idx, 'text']
    print(f"Texto ({len(text)} chars):")
    print(f"{text[:300]}{'...' if len(text) > 300 else ''}\n")
    
    # Mostrar entidades por modelo
    for model in ['bert', 'biobert', 'spacy']:
        entities = df_results.loc[idx, f'entities_{model}']
        if entities:
            print(f"\n{model.upper()} ({len(entities)} entidades):")
            for start, end, label, text_ent in entities[:5]:  # Mostrar máximo 5
                print(f"  [{label}] {text_ent} (pos {start}-{end})")
            if len(entities) > 5:
                print(f"  ... y {len(entities)-5} más")
        else:
            print(f"\n{model.upper()}: Sin entidades detectadas")

print(f"\n{'='*80}")
print("ESTADÍSTICAS GLOBALES")
print(f"{'='*80}\n")

# Estadísticas por modelo
for model in ['bert', 'biobert', 'spacy']:
    col = f'entities_{model}'
    textos_con_entidades = (df_results[col].str.len() > 0).sum()
    total_entidades = df_results[col].apply(len).sum()
    promedio = total_entidades / len(df_results) if len(df_results) > 0 else 0
    
    print(f"{model.upper()}:")
    print(f"  Textos con entidades: {textos_con_entidades} ({textos_con_entidades/len(df_results)*100:.1f}%)")
    print(f"  Total entidades: {total_entidades}")
    print(f"  Promedio por texto: {promedio:.2f}")
    
    # Top etiquetas
    all_labels = []
    for entities in df_results[col]:
        all_labels.extend([label for _, _, label, _ in entities])
    
    if all_labels:
        top_labels = Counter(all_labels).most_common(10)
        print(f"  Top 5 etiquetas:")
        for label, count in top_labels[:5]:
            print(f"    - {label}: {count} ({count/len(all_labels)*100:.1f}%)")
    print()

# Comparación: textos donde cada modelo detectó algo diferente
print(f"{'='*80}")
print("COBERTURA COMPARADA")
print(f"{'='*80}\n")

df_results['has_bert'] = df_results['entities_bert'].str.len() > 0
df_results['has_biobert'] = df_results['entities_biobert'].str.len() > 0
df_results['has_spacy'] = df_results['entities_spacy'].str.len() > 0

print(f"Textos donde SOLO detectó:")
print(f"  BERT: {(df_results['has_bert'] & ~df_results['has_biobert'] & ~df_results['has_spacy']).sum()}")
print(f"  BioBERT: {(df_results['has_biobert'] & ~df_results['has_bert'] & ~df_results['has_spacy']).sum()}")
print(f"  spaCy: {(df_results['has_spacy'] & ~df_results['has_bert'] & ~df_results['has_biobert']).sum()}")

print(f"\nTextos donde TODOS detectaron algo: {(df_results['has_bert'] & df_results['has_biobert'] & df_results['has_spacy']).sum()}")
print(f"Textos donde NINGUNO detectó: {(~df_results['has_bert'] & ~df_results['has_biobert'] & ~df_results['has_spacy']).sum()}")

# Análisis de longitud de texto vs detección
print(f"\n{'='*80}")
print("RELACIÓN LONGITUD-DETECCIÓN")
print(f"{'='*80}\n")

df_results['text_length'] = df_results['text'].str.len()
print(f"Longitud promedio texto: {df_results['text_length'].mean():.0f} chars")
print(f"Textos truncados (>2000 chars): {(df_results['text_original_length'] > 2000).sum()}")

for model in ['bert', 'biobert', 'spacy']:
    col = f'entities_{model}'
    print(f"\n{model.upper()}:")
    print(f"  Textos con entidades: {(df_results[col].str.len() > 0).sum()}")
    print(f"  Total entidades: {df_results[col].apply(len).sum()}")
    
    df_with_ents = df_results[df_results[col].str.len() > 0]
    if len(df_with_ents) > 0:
        avg_length = df_with_ents['text_length'].mean()
        print(f"{model}: textos con entidades tienen promedio {avg_length:.0f} chars")
        
        



Total de textos procesados: 183744

EJEMPLO 0
Texto (740 chars):
likely frontpage subreddit glad exist want reassurance people afraid needle go colonoscopy pretty young like needle like hospital dislike combine aside spend 3 day foreign hospital week get test afraid go ulcerative colitis understand simliar crohns disease mother condition gi problem bring hospital...


BERT: Sin entidades detectadas

BIOBERT (9 entidades):
  [0] likely frontpage subreddit glad exist want reassurance people afraid needle go colonoscopy pretty young like needle like hospital dislike combine aside spend 3 day foreign hospital week get test afraid go (pos 0-204)
  [DISEASE] ulcerative colitis (pos 205-223)
  [0] understand simliar (pos 224-242)
  [DISEASE] crohns disease (pos 243-257)
  [0] mother condition gi problem bring hospital worried colonoscopy horrific experience bad procedure far have prep drink ton 4 liter laxative laced water clear entire gi tract procedure pleasant procedure iv sedation make co

In [37]:
#Comparacion de modelos NER: BERT, BioBERT, spaCy, scispaCy, con el cuerpo de texto truncado a 2000 caracteres para evitar errores de longitud. NO LEMATIZADO

# Comparación: textos donde cada modelo usando el texto original sin lematizar.
# El lematizado puede afectar la detección de entidades.(pantallazo), por lo que vamos a ver si mejora el reconocimiento de entidades en bert y biobert
# Límite arquitectónico de BERT:necesario truncar a ~512 tokens (2000 chars aprox)

import torch
import json
from transformers import pipeline
import spacy
import numpy as np
import pandas as pd
from collections import Counter
from tqdm import tqdm

print(f"✅ PyTorch: {torch.__version__}")

# Modelos
HF_MODELS = {
    "bert": "dslim/bert-base-NER",
    #"biobert": "dmis-lab/biobert-v1.1",  # BioBERT base (sin fine-tune específico NER)
    "biobert": "alvaroalon2/biobert_diseases_ner"  # Fine-tuned en enfermedades
    #"biobert": "siddharthtumre/biobert-finetuned-ner"
}


# spaCy
nlp_spacy = spacy.load("en_core_web_sm")
nlp_scispacy = None  # Omitir por ahora

# Helpers
def normalize_hf(ner_out):
    spans = []
    for ent in ner_out:
        label = ent.get("entity_group") or ent.get("entity")
        spans.append((ent["start"], ent["end"], label, ent["word"]))
    return spans

def normalize_spacy(doc):
    return [(ent.start_char, ent.end_char, ent.label_, ent.text) for ent in doc.ents]

def truncate_text(text, max_chars=2000):
    """Trunca a ~512 tokens (aprox 2000 chars)"""
    return text[:max_chars] if len(text) > max_chars else text

# Cargar datos
df = pd.read_csv('/Users/fjosesala/Documents/GitHub/IBD-NLP-RiskPrediction/data/processed/reddit_ibd_preprocessed.csv')
texts = df["selftext_clean"].fillna("").astype(str).tolist()
ids = df.get("id", pd.Series(range(len(df)))).tolist()

print(f"Total textos: {len(texts)}")

# Crear pipelines (sin truncation en constructor)
hf_pipes = {}
for name, model_name in HF_MODELS.items():
    print(f"Cargando {name}...")
    hf_pipes[name] = pipeline("ner", model=model_name, tokenizer=model_name, grouped_entities=True, device=-1)

print("✅ Modelos cargados\n")

# Procesar (prueba con 100 primero)
SAMPLE = len(texts)
idxs = list(range(min(SAMPLE, len(texts))))
results = []

for i in tqdm(idxs, desc="NER"):
    text_orig = texts[i]
    text = truncate_text(text_orig, max_chars=2000)  # ← Trunca aquí
    
    row = {"id": ids[i], "text": text, "text_original_length": len(text_orig)}
    
    # HF models
    for name, pipe in hf_pipes.items():
        try:
            out = pipe(text)
            row[f"entities_{name}"] = normalize_hf(out)
        except Exception as e:
            print(f"⚠️  Error {name} en texto {i}: {str(e)[:50]}")
            row[f"entities_{name}"] = []
    
    # spaCy (también puede fallar con textos largos)
    try:
        if len(text) < 1000000:  # spaCy tiene límite 1M chars
            doc_sp = nlp_spacy(text)
            row["entities_spacy"] = normalize_spacy(doc_sp)
        else:
            row["entities_spacy"] = []
    except Exception as e:
        print(f"⚠️  Error spaCy en texto {i}")
        row["entities_spacy"] = []
    
    # scispaCy (omitir por ahora)
    row["entities_scispacy"] = []
    
    results.append(row)

# Guardar
out_df = pd.DataFrame(results)

import os
os.makedirs("./data/processed", exist_ok=True)

out_df.to_pickle("./data/processed/ner_comparisons_WO_LEMMA.pkl")
out_df.to_csv("./data/processed/ner_comparisons_preview_WO_LEMMA.csv", index=False)

print(f"\n✅ Procesados {len(results)} textos")
print(f"📁 Archivos guardados en ./data/processed/")

# Resumen rápido
for model in ["bert", "biobert", "spacy"]:
    col = f"entities_{model}"
    total = out_df[col].apply(len).sum()
    print(f"  {model}: {total} entidades detectadas")


✅ PyTorch: 2.9.0
Total textos: 183744
Cargando bert...


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
/Users/fjosesala/Documents/GitHub/IBD-NLP-RiskPrediction/.venv/lib/python3.13/site-packages/transformers/pipelines/token_classification.py:186: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.wa

Cargando biobert...


Device set to use cpu


✅ Modelos cargados



NER: 100%|██████████| 183744/183744 [6:19:35<00:00,  8.07it/s]  



✅ Procesados 183744 textos
📁 Archivos guardados en ./data/processed/
  bert: 3838 entidades detectadas
  biobert: 1007465 entidades detectadas
  spacy: 571480 entidades detectadas


In [39]:
# Checkear resultados generados por cada modelo NER

import pandas as pd
from collections import Counter

# Cargar resultados
df_results = pd.read_pickle("./data/processed/ner_comparisons_WO_LEMMA.pkl")

print(f"Total de textos procesados: {len(df_results)}")
print(f"="*80)

# Ver varios ejemplos para comparar
for idx in [0, 1, 2, 4, 10]:
    print(f"\n{'='*80}")
    print(f"EJEMPLO {idx}")
    print(f"{'='*80}")
    
    text = df_results.loc[idx, 'text']
    print(f"Texto ({len(text)} chars):")
    print(f"{text[:300]}{'...' if len(text) > 300 else ''}\n")
    
    # Mostrar entidades por modelo
    for model in ['bert', 'biobert', 'spacy']:
        entities = df_results.loc[idx, f'entities_{model}']
        if entities:
            print(f"\n{model.upper()} ({len(entities)} entidades):")
            for start, end, label, text_ent in entities[:5]:  # Mostrar máximo 5
                print(f"  [{label}] {text_ent} (pos {start}-{end})")
            if len(entities) > 5:
                print(f"  ... y {len(entities)-5} más")
        else:
            print(f"\n{model.upper()}: Sin entidades detectadas")

print(f"\n{'='*80}")
print("ESTADÍSTICAS GLOBALES")
print(f"{'='*80}\n")

# Estadísticas por modelo
for model in ['bert', 'biobert', 'spacy']:
    col = f'entities_{model}'
    textos_con_entidades = (df_results[col].str.len() > 0).sum()
    total_entidades = df_results[col].apply(len).sum()
    promedio = total_entidades / len(df_results) if len(df_results) > 0 else 0
    
    print(f"{model.upper()}:")
    print(f"  Textos con entidades: {textos_con_entidades} ({textos_con_entidades/len(df_results)*100:.1f}%)")
    print(f"  Total entidades: {total_entidades}")
    print(f"  Promedio por texto: {promedio:.2f}")
    
    # Top etiquetas
    all_labels = []
    for entities in df_results[col]:
        all_labels.extend([label for _, _, label, _ in entities])
    
    if all_labels:
        top_labels = Counter(all_labels).most_common(10)
        print(f"  Top 5 etiquetas:")
        for label, count in top_labels[:5]:
            print(f"    - {label}: {count} ({count/len(all_labels)*100:.1f}%)")
    print()

# Comparación: textos donde cada modelo detectó algo diferente
print(f"{'='*80}")
print("COBERTURA COMPARADA")
print(f"{'='*80}\n")

df_results['has_bert'] = df_results['entities_bert'].str.len() > 0
df_results['has_biobert'] = df_results['entities_biobert'].str.len() > 0
df_results['has_spacy'] = df_results['entities_spacy'].str.len() > 0

print(f"Textos donde SOLO detectó:")
print(f"  BERT: {(df_results['has_bert'] & ~df_results['has_biobert'] & ~df_results['has_spacy']).sum()}")
print(f"  BioBERT: {(df_results['has_biobert'] & ~df_results['has_bert'] & ~df_results['has_spacy']).sum()}")
print(f"  spaCy: {(df_results['has_spacy'] & ~df_results['has_bert'] & ~df_results['has_biobert']).sum()}")

print(f"\nTextos donde TODOS detectaron algo: {(df_results['has_bert'] & df_results['has_biobert'] & df_results['has_spacy']).sum()}")
print(f"Textos donde NINGUNO detectó: {(~df_results['has_bert'] & ~df_results['has_biobert'] & ~df_results['has_spacy']).sum()}")

# Análisis de longitud de texto vs detección
print(f"\n{'='*80}")
print("RELACIÓN LONGITUD-DETECCIÓN")
print(f"{'='*80}\n")

df_results['text_length'] = df_results['text'].str.len()
print(f"Longitud promedio texto: {df_results['text_length'].mean():.0f} chars")
print(f"Textos truncados (>2000 chars): {(df_results['text_original_length'] > 2000).sum()}")

for model in ['bert', 'biobert', 'spacy']:
    col = f'entities_{model}'
    print(f"\n{model.upper()}:")
    print(f"  Textos con entidades: {(df_results[col].str.len() > 0).sum()}")
    print(f"  Total entidades: {df_results[col].apply(len).sum()}")
    
    df_with_ents = df_results[df_results[col].str.len() > 0]
    if len(df_with_ents) > 0:
        avg_length = df_with_ents['text_length'].mean()
        print(f"{model}: textos con entidades tienen promedio {avg_length:.0f} chars")
        

Total de textos procesados: 183744

EJEMPLO 0
Texto (1386 chars):
i am not likely to frontpage this subreddit but i am glad it exists i did however want to be of some reassurance to people who are afraid of needles or going in for a colonoscopy now i am pretty young i do not like needles and i do not like hospitals but all of my dislikes combined aside i had to sp...


BERT: Sin entidades detectadas

BIOBERT (11 entidades):
  [0] i am not likely to frontpage this subreddit but i am glad it exists i did however want to be of some reassurance to people who are afraid of needles or going in for a colonoscopy now i am pretty young i do not like needles and i do not like hospitals but all of my dislikes combined aside i had to spend 3 days in a foreign hospital this week getting tested for what i was afraid was going to be (pos 0-395)
  [DISEASE] ulcerative colitis (pos 396-414)
  [0] what i understand to be simliar to (pos 415-449)
  [DISEASE] crohns disease (pos 450-464)
  [0] my mother h

Total de textos procesados: 183744


BERT
Total entidades detectadas: 3838

Top 20 entidades más frecuentes:
    354x  s
    312x  me
    189x  x
    166x  ##dic
    158x  g
    129x  ##tel
    125x  ##dicaid
    117x  ##dicare
    113x  xel
    112x  ##dia
     98x  can
     93x  am
     69x  as
     69x  stel
     67x  i
     48x  j
     45x  ##nese
     45x  c
     43x  ##el
     40x  hum

BIOBERT
Total entidades detectadas: 1007465

Top 20 entidades más frecuentes:
  43027x  pain
  17173x  crohn s
  16245x  deleted
  14681x  inflammation
  13066x  c
  12838x  crohns
  12721x  and
  11855x  diarrhea
   7211x  i
   6210x  nausea
   6184x  bleeding
   6075x  uc
   6000x  ulcerative colitis
   5695x  crohn s disease
   5458x  colitis
   5346x  anxiety
   5312x  fatigue
   4840x  fistula
   4827x  removed
   4636x  constipation

SPACY
Total entidades detectadas: 571480

Top 20 entidades más frecuentes:
  26676x  first
  14864x  today
  10626x  one
   8611x  2
   8007x  two
   6789x  3
 

In [1]:
import spacy
nlp_bc5 = spacy.load("en_ner_bc5cdr_md")
print("OK:", nlp_bc5.meta.get('name'))


OK: ner_bc5cdr_md


/Users/fjosesala/Documents/GitHub/IBD-NLP-RiskPrediction/.venv311/lib/python3.11/site-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


In [1]:
#abrir reddit_ibd_preprocessed
import pandas as pd
df = pd.read_csv('/Users/fjosesala/Documents/GitHub/IBD-NLP-RiskPrediction/data/processed/reddit_ibd_preprocessed.csv')

#exportar las 10 primeras filas a un csv
df.head(10).to_csv('/Users/fjosesala/Documents/GitHub/IBD-NLP-RiskPrediction/data/processed/reddit_ibd_preprocessed_preview.csv', index=False)



In [4]:
# BC5CDR directo desde CSV original con lector manual

import os
import csv
import sys
import spacy
import pandas as pd
from tqdm import tqdm
from collections import Counter

# -----------------------
# Configuración
# -----------------------
# Asegúrate de actualizar esta ruta a tu archivo de entrada
INPUT_CSV_ORIG = '/Users/fjosesala/Documents/GitHub/IBD-NLP-RiskPrediction/data/processed/reddit_ibd_preprocessed.csv'
OUTPUT_DIR = './data/processed'
OUTPUT_CSV = os.path.join(OUTPUT_DIR, 'ner_scispacy_bc5cdr.csv')
OUTPUT_LONG = os.path.join(OUTPUT_DIR, 'ner_scispacy_bc5cdr_long.csv')

MAX_CHARS = 2000

# -----------------------
# Utilidades
# -----------------------
def truncate_text(text, max_chars=2000):
    """Trunca el texto a un número máximo de caracteres."""
    text = text or ""
    return text[:max_chars] if len(text) > max_chars else text

def ensure_dir(path):
    """Asegura que un directorio exista."""
    os.makedirs(path, exist_ok=True)

# -----------------------
# Leer CSV original manualmente (evita pandas parser)
# -----------------------
# Aumentar el límite del tamaño del campo para textos largos
csv.field_size_limit(min(sys.maxsize, 10_000_000))

KEEP = {'id','title','selftext'}
rows_data = []

print("Leyendo CSV original con lector manual...")
try:
    with open(INPUT_CSV_ORIG, 'r', encoding='utf-8', errors='replace', newline='') as f:
        # Sniff para detectar el dialecto (separador, comillas)
        sniffer = csv.Sniffer()
        sample = f.read(8192) # Leer una muestra
        f.seek(0) # Volver al inicio
        try:
            dialect = sniffer.sniff(sample)
        except Exception:
            # Fallback a un dialecto estándar si falla el sniff
            class Dialect(csv.Dialect):
                delimiter = ','
                quotechar = '"'
                escapechar = None
                doublequote = True
                lineterminator = '\n'
                quoting = csv.QUOTE_MINIMAL
            dialect = Dialect()

        reader = csv.DictReader(f, dialect=dialect)
        
        for row in reader:
            try:
                post_id = row.get('id', '')
                title = row.get('title', '')
                selftext = row.get('selftext', '')

                # Concatenar title + selftext
                full_text = (title + ' ' + selftext).strip()
                text_trunc = truncate_text(full_text, MAX_CHARS)

                rows_data.append({'id': post_id, 'text': text_trunc})
            except Exception as e_row:
                # Omitir filas problemáticas
                # print(f"Error procesando fila: {e_row}")
                continue

    print(f"✅ Leídas {len(rows_data)} filas.\n")

except FileNotFoundError:
    print(f"🚨 ERROR: No se encontró el archivo de entrada en: {INPUT_CSV_ORIG}")
    print("Por favor, actualiza la variable INPUT_CSV_ORIG en el script.")
    sys.exit(1) # Salir si no se encuentra el archivo
except Exception as e_file:
    print(f"🚨 ERROR Inesperado leyendo el archivo: {e_file}")
    sys.exit(1)


# -----------------------
# Cargar scispaCy BC5CDR
# -----------------------
print("Cargando scispaCy en_ner_bc5cdr_md...")
try:
    nlp_bc5 = spacy.load("en_ner_bc5cdr_md")
    print("✅ scispaCy BC5CDR cargado.\n")
except IOError:
    print("🚨 ERROR: Modelo 'en_ner_bc5cdr_md' no encontrado.")
    print("Por favor, instálalo ejecutando:")
    print("pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_ner_bc5cdr_md-0.5.1.tar.gz")
    sys.exit(1)

# -----------------------
# Ejecutar NER con BC5CDR (solo DISEASE y CHEMICAL)
# -----------------------
results = []
print("Procesando textos con scispaCy BC5CDR...")
for item in tqdm(rows_data, desc="scispaCy BC5CDR"):
    doc = nlp_bc5(item['text'])
    
    # Extraer solo las entidades de interés
    ents = [(e.start_char, e.end_char, e.label_, e.text)
            for e in doc.ents if e.label_ in {"DISEASE", "CHEMICAL"}]
    
    results.append({
        "id": item['id'],
        "text": item['text'],
        "entities_scispacy_bc5cdr": str(ents), # serializar como string para CSV
        "n_entities": len(ents)
    })

# -----------------------
# Guardar resultados (solo CSV)
# -----------------------
ensure_dir(OUTPUT_DIR)
out_df = pd.DataFrame(results)
out_df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8')

print(f"\n✅ Procesados {len(out_df)} textos")
print(f"📁 Guardado CSV: {OUTPUT_CSV}")

# -----------------------
# Exportar formato largo y resumen
# -----------------------
long_rows = []
for _, r in out_df.iterrows():
    import ast
    try:
        # Convertir el string de vuelta a lista de tuplas
        ents = ast.literal_eval(r['entities_scispacy_bc5cdr'])
    except Exception:
        ents = [] # Manejar casos donde la evaluación falla

    for (s, e, l, t) in ents:
        long_rows.append({
            "id": r["id"],
            "label": l,
            "entity_text": t,
            "start": s,
            "end": e
        })

df_long = pd.DataFrame(long_rows)
df_long.to_csv(OUTPUT_LONG, index=False, encoding='utf-8')
print(f"📁 Guardado CSV (largo): {OUTPUT_LONG}\n")

# Mostrar resumen si se encontraron entidades
if not df_long.empty:
    print("="*80)
    print("RESUMEN DE ENTIDADES")
    print("="*80)

    total_disease = (df_long['label'] == 'DISEASE').sum()
    total_chemical = (df_long['label'] == 'CHEMICAL').sum()
    
    print(f"\nTotal DISEASE: {total_disease}")
    print(f"Total CHEMICAL: {total_chemical}")

    print("\nTop 20 DISEASE:")
    for ent, cnt in Counter(df_long[df_long['label'] == 'DISEASE']['entity_text']).most_common(20):
        print(f" {cnt:>6}x {ent}")

    print("\nTop 20 CHEMICAL (fármacos):")
    for ent, cnt in Counter(df_long[df_long['label'] == 'CHEMICAL']['entity_text']).most_common(20):
        print(f" {cnt:>6}x {ent}")
else:
    print("\n⚠️ No se detectaron entidades DISEASE/CHEMICAL.")

print("\n" + "="*80)
print("✅ Pipeline completado.")
print("="*80)





Leyendo CSV original con lector manual...
✅ Leídas 281414 filas.

Cargando scispaCy en_ner_bc5cdr_md...
✅ scispaCy BC5CDR cargado.

Procesando textos con scispaCy BC5CDR...


scispaCy BC5CDR: 100%|██████████| 281414/281414 [1:51:12<00:00, 42.17it/s]  



✅ Procesados 281414 textos
📁 Guardado CSV: ./data/processed/ner_scispacy_bc5cdr.csv
📁 Guardado CSV (largo): ./data/processed/ner_scispacy_bc5cdr_long.csv

RESUMEN DE ENTIDADES

Total DISEASE: 524515
Total CHEMICAL: 247373

Top 20 DISEASE:
  62021x pain
  22584x UC
  19530x inflammation
  18507x diarrhea
  17067x IBD
  12611x crohns
   9600x Crohns
   8492x bleeding
   8067x anxiety
   7650x fatigue
   7527x colitis
   6418x nausea
   6334x Crohn’s
   6136x ulcerative colitis
   5481x constipation
   5250x fistula
   4955x abdominal pain
   4827x IBS
   3993x pains
   3886x infection

Top 20 CHEMICAL (fármacos):
  21771x prednisone
  13226x Remicade
  10316x humira
   9779x remicade
   9269x mesalamine
   8907x steroids
   6787x ’ll
   6622x Prednisone
   5742x UC
   5388x calprotectin
   4504x iron
   4351x budesonide
   3244x azathioprine
   3144x Mesalamine
   3084x steroid
   2954x alcohol
   2791x Hi
   2189x mesalazine
   2120x Imuran
   2095x Lialda

✅ Pipeline completado.


## Ver Datasets Generados

In [2]:
ibd_entinties_scispaCy = pd.read_csv('/Users/fjosesala/Documents/GitHub/IBD-NLP-RiskPrediction/data/processed/ner_scispacy_bc5cdr.csv')
ibd_entinties_scispaCy.head()

,id,text,entities_scispacy_bc5cdr,n_entities
0,dflwn,Don't be afraid of diagnostic procedures.... I...,"[(451, 469, 'DISEASE', 'ulcerative colitis'), ...",4
1,dfyy1,Mayo Clinic article on Crohn's Disease. Mayo C...,"[(23, 62, 'DISEASE', ""Crohn's Disease. Mayo Cl...",2
2,dfz1m,Has anyone else here taken Remicade? What did...,"[(27, 35, 'CHEMICAL', 'Remicade'), (105, 113, ...",2
3,dh6zd,"I was Diagnosed With Crohns Disease at age 9 ,...","[(21, 35, 'DISEASE', 'Crohns Disease')]",1
4,- Was diagnosed 10 years ago (at age 10) and w...,"coincidentally I started smoking...""",[],0


In [3]:
ibd_entities_biobert = pd.read_csv('/Users/fjosesala/Documents/GitHub/IBD-NLP-RiskPrediction/notebooks/data/processed/ner_comparisons_preview.csv')
ibd_entities_biobert.head()

,id,text,text_original_length,entities_bert,entities_biobert,entities_spacy,entities_scispacy
0,dflwn,likely frontpage subreddit glad exist want rea...,740,[],"[(0, 204, '0', 'likely frontpage subreddit gla...","[(158, 163, 'DATE', '3 day'), (369, 378, 'PERS...",[]
1,dfyy1,mayo clinic article crohn s disease,35,[],"[(0, 19, '0', 'mayo clinic article'), (20, 35,...","[(0, 27, 'ORG', 'mayo clinic article crohn s')]",[]
2,dfz1m,take remicade think help,24,[],"[(0, 24, '0', 'take remicade think help')]",[],[]
3,dh6zd,like title say ama,18,[],"[(0, 18, '0', 'like title say ama')]",[],[]
4,dhgot,small bowel resection appendectomy 6 year ago ...,139,[],"[(0, 101, '0', 'small bowel resection appendec...","[(35, 45, 'DATE', '6 year ago')]",[]


In [4]:
# Pegar el el scypacy con los otros resultados
ibd_merged = ibd_entities_biobert.merge(ibd_entinties_scispaCy[['id', 'entities_scispacy_bc5cdr']], on='id', how='left')
#eliminar columna entities_scispacy
ibd_merged = ibd_merged.drop(columns=['entities_scispacy', 'text_original_length', "entities_biobert"])#se remueve biobert... pues no aporta mucho, muchos 0s


ibd_merged.head()

,id,text,entities_bert,entities_spacy,entities_scispacy_bc5cdr
0,dflwn,likely frontpage subreddit glad exist want rea...,[],"[(158, 163, 'DATE', '3 day'), (369, 378, 'PERS...","[(451, 469, 'DISEASE', 'ulcerative colitis'), ..."
1,dfyy1,mayo clinic article crohn s disease,[],"[(0, 27, 'ORG', 'mayo clinic article crohn s')]","[(23, 62, 'DISEASE', ""Crohn's Disease. Mayo Cl..."
2,dfz1m,take remicade think help,[],[],"[(27, 35, 'CHEMICAL', 'Remicade'), (105, 113, ..."
3,dh6zd,like title say ama,[],[],"[(21, 35, 'DISEASE', 'Crohns Disease')]"
4,dhgot,small bowel resection appendectomy 6 year ago ...,[],"[(35, 45, 'DATE', '6 year ago')]","[(66, 87, 'DISEASE', 'small bowel resection'),..."


In [5]:
#Remueve de la columna 'entities_scispacy_bc5cdr' las entidades que se dieron por error: Hi, ’ll, UC

def remove_erroneous_entities(entities_str):
    import ast
    try:
        entities = ast.literal_eval(entities_str)
        cleaned = [ent for ent in entities if ent[3] not in {'Hi', '’ll', 'UC'}]
        return str(cleaned)
    except Exception:
        return '[]'  # Retornar lista vacía en caso de error
    
    
ibd_merged['entities_scispacy_bc5cdr'] = ibd_merged['entities_scispacy_bc5cdr'].apply(remove_erroneous_entities)

In [6]:
# Realizar conteo de entidades, por modelo
import ast

ibd_merged['n_entities_scispacy_bc5cdr'] = ibd_merged['entities_scispacy_bc5cdr'].apply(lambda x: len(ast.literal_eval(x)))
ibd_merged['n_entities_spacy'] = ibd_merged['entities_spacy'].apply(lambda x: len(ast.literal_eval(x)))
ibd_merged['n_entities_bert'] = ibd_merged['entities_bert'].apply(lambda x: len(ast.literal_eval(x)))

ibd_merged

,id,text,entities_bert,entities_spacy,entities_scispacy_bc5cdr,n_entities_scispacy_bc5cdr,n_entities_spacy,n_entities_bert
0,dflwn,likely frontpage subreddit glad exist want rea...,[],"[(158, 163, 'DATE', '3 day'), (369, 378, 'PERS...","[(451, 469, 'DISEASE', 'ulcerative colitis'), ...",4,4,0
1,dfyy1,mayo clinic article crohn s disease,[],"[(0, 27, 'ORG', 'mayo clinic article crohn s')]","[(23, 62, 'DISEASE', ""Crohn's Disease. Mayo Cl...",2,1,0
2,dfz1m,take remicade think help,[],[],"[(27, 35, 'CHEMICAL', 'Remicade'), (105, 113, ...",2,0,0
3,dh6zd,like title say ama,[],[],"[(21, 35, 'DISEASE', 'Crohns Disease')]",1,0,0
4,dhgot,small bowel resection appendectomy 6 year ago ...,[],"[(35, 45, 'DATE', '6 year ago')]","[(66, 87, 'DISEASE', 'small bowel resection'),...",2,1,0
...,...,...,...,...,...,...,...,...
183740,15wpzxc,option 1 step 1 send email info geronova com m...,[],"[(7, 8, 'CARDINAL', '1'), (14, 15, 'CARDINAL',...",[],0,9,0
183741,15vva9w,ulcerative colitis comprehensive evidence base...,[],[],"[(0, 18, 'DISEASE', 'Ulcerative Colitis'), (76...",2,0,0
183742,15vv9ts,jay pravda md mph ulcerative colitis explain,[],"[(0, 10, 'PERSON', 'jay pravda'), (11, 17, 'PE...","[(14, 17, 'CHEMICAL', 'MPH'), (32, 50, 'DISEAS...",4,2,0
183743,15vv9gl,jay pravda md mph ulcerative colitis cure clin...,[],"[(0, 10, 'PERSON', 'jay pravda'), (11, 17, 'PE...","[(14, 36, 'CHEMICAL', 'MPH Ulcerative colitis'...",2,2,0


In [7]:
import re
import pandas as pd

# Asegura que 'text' sea string y sin NaN
ibd_merged['text'] = ibd_merged['text'].astype(str).fillna('')

SYMPTOMS = {
    'has_blood': r'\b(bloody (stool|stools)|blood in (stool|toilet)|rectal bleeding|blood on (toilet paper|paper)|blood when (wiping|i wipe))\b',
    'has_diarrhea': r'\b(diarrh(oe)a|diarrhea|loose stools?|watery stools?|bowel movements? (are )?(loose|watery))\b',
    'has_pain': r'\b(abdominal pain|stomach pain|belly pain|cramps?|cramping|severe pain|sharp pain)\b',
    'has_fever': r'\b(fever|pyrexia|running a fever|high temperature)\b',
    'has_weight_loss': r'\b(weight loss|lost (a lot of )?weight|losing weight)\b',
}
RECENT = r'\b(today|yesterday|last (\d+\s)?(day|days|week|weeks)|past (few )?(days?|weeks?)|this (week|month)|since (yesterday|last night|monday))\b'
ER = r'\b(er|emergency room|emergency dept|ed visit|urgent care|went to (the )?er|hospital(ization)?|admitted|inpatient)\b'
PROC = r'\b(colonoscopy|endoscopy|sigmoidoscopy|biopsy|resection|colectomy|ileostomy|colostomy|strictureplasty)\b'

def flags_from_text_safe(t):
    # Tolerante a NaN/objetos
    t = '' if (t is None or (isinstance(t, float) and pd.isna(t))) else str(t)
    t = t.lower()
    out = {}
    for k, pat in SYMPTOMS.items():
        out[k] = int(re.search(pat, t) is not None)
    out['n_symptom'] = out['has_blood'] + out['has_diarrhea'] + out['has_pain'] + out['has_fever'] + out['has_weight_loss']
    out['temporal_recent'] = int(re.search(RECENT, t) is not None)
    out['has_er'] = int(re.search(ER, t) is not None)
    out['has_procedure'] = int(re.search(PROC, t) is not None)
    return pd.Series(out, dtype='int64')

df_flags = ibd_merged[['id','text']].copy()
df_flags = df_flags.join(df_flags['text'].apply(flags_from_text_safe))




In [9]:
df_flags.head(10) 

,id,text,has_blood,has_diarrhea,has_pain,has_fever,has_weight_loss,n_symptom,temporal_recent,has_er,has_procedure
0,dflwn,likely frontpage subreddit glad exist want rea...,0,0,0,0,0,0,0,1,1
1,dfyy1,mayo clinic article crohn s disease,0,0,0,0,0,0,0,0,0
2,dfz1m,take remicade think help,0,0,0,0,0,0,0,0,0
3,dh6zd,like title say ama,0,0,0,0,0,0,0,0,0
4,dhgot,small bowel resection appendectomy 6 year ago ...,0,0,0,0,0,0,0,0,1
5,diwa3,hi 8 year diagnose cd remission incident obstr...,0,0,0,0,0,0,0,1,0
6,dkeom,feel like good job deal physical pain loss wei...,0,0,0,0,0,0,0,0,0
7,dlt0i,think crohn s disease lifetime drug surgery th...,0,0,0,0,0,0,0,0,0
8,dpm24,recently diagnose crohn s exactly time unende ...,0,1,0,0,0,1,0,0,0
9,dsru9,miracle mineral solution cure crohn s disease ...,0,0,0,0,0,0,0,0,0


In [13]:
# merge con ibd_merged
ibd_final = ibd_merged.merge(df_flags, on='id', how='left')
#eliminar text_y
ibd_final = ibd_final.drop(columns=['text_y'])

In [15]:
ibd_final.head(10)

,id,text_x,entities_bert,entities_spacy,entities_scispacy_bc5cdr,n_entities_scispacy_bc5cdr,n_entities_spacy,n_entities_bert,has_blood,has_diarrhea,has_pain,has_fever,has_weight_loss,n_symptom,temporal_recent,has_er,has_procedure
0,dflwn,likely frontpage subreddit glad exist want rea...,[],"[(158, 163, 'DATE', '3 day'), (369, 378, 'PERS...","[(451, 469, 'DISEASE', 'ulcerative colitis'), ...",4,4,0,0,0,0,0,0,0,0,1,1
1,dfyy1,mayo clinic article crohn s disease,[],"[(0, 27, 'ORG', 'mayo clinic article crohn s')]","[(23, 62, 'DISEASE', ""Crohn's Disease. Mayo Cl...",2,1,0,0,0,0,0,0,0,0,0,0
2,dfz1m,take remicade think help,[],[],"[(27, 35, 'CHEMICAL', 'Remicade'), (105, 113, ...",2,0,0,0,0,0,0,0,0,0,0,0
3,dh6zd,like title say ama,[],[],"[(21, 35, 'DISEASE', 'Crohns Disease')]",1,0,0,0,0,0,0,0,0,0,0,0
4,dhgot,small bowel resection appendectomy 6 year ago ...,[],"[(35, 45, 'DATE', '6 year ago')]","[(66, 87, 'DISEASE', 'small bowel resection'),...",2,1,0,0,0,0,0,0,0,0,0,1
5,diwa3,hi 8 year diagnose cd remission incident obstr...,[],"[(3, 9, 'DATE', '8 year'), (62, 67, 'DATE', '3...","[(0, 5, 'DISEASE', 'Itchy'), (242, 252, 'DISEA...",5,4,0,0,0,0,0,0,0,0,1,0
6,dkeom,feel like good job deal physical pain loss wei...,[],"[(63, 70, 'PERSON', 'crohn s')]","[(40, 62, 'DISEASE', ""Crohn's? I feel like I"")...",4,1,0,0,0,0,0,0,0,0,0,0
7,dlt0i,think crohn s disease lifetime drug surgery th...,[],"[(6, 13, 'PERSON', 'crohn s')]","[(6, 21, 'DISEASE', ""Crohn's disease""), (110, ...",2,1,0,0,0,0,0,0,0,0,0,0
8,dpm24,recently diagnose crohn s exactly time unende ...,[],"[(66, 74, 'MONEY', '2 dollar'), (412, 414, 'CA...","[(25, 63, 'DISEASE', ""Crohn's? I was recently ...",9,2,0,0,1,0,0,0,1,0,0,0
9,dsru9,miracle mineral solution cure crohn s disease ...,[],"[(57, 68, 'DATE', '15 year old')]","[(43, 133, 'DISEASE', 'Crohn\'s \r\nDisease, o...",2,1,0,0,0,0,0,0,0,0,0,0


In [14]:
#exportar ibd_final a csv
ibd_final.to_csv('/Users/fjosesala/Documents/GitHub/IBD-NLP-RiskPrediction/data/processed/ibd_ner_features_final.csv', index=False)